__DO NOT USE THIS NOTEBOOK!__

Use [this](Spectra_comparison_tran_small_signal_Sardinia.ipynb) instead.

In [ ]:
import os
import sys
from tqdm.notebook import tqdm
import numpy as np
from scipy.signal import lti, lsim, welch
if not '..' in sys.path:
    sys.path.append('..')
from pfcommon import OU_2, combine_output_spectra

dB = 10

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import FixedLocator, NullLocator, FixedFormatter
import seaborn as sns

fontsize = 9
lw = 0.75
matplotlib.rc('font', **{'family': 'Times', 'size': fontsize})
matplotlib.rc('axes', **{'linewidth': 0.75, 'labelsize': fontsize})
matplotlib.rc('xtick', **{'labelsize': fontsize})
matplotlib.rc('ytick', **{'labelsize': fontsize})
matplotlib.rc('xtick.major', **{'width': lw, 'size':3})
matplotlib.rc('ytick.major', **{'width': lw, 'size':3})
matplotlib.rc('ytick.minor', **{'width': lw, 'size':1.5})

Load the data:

In [ ]:
N = 100
data_file = os.path.join('..','data','Sardinia','SM_configs_from_data','001',
                         'V2020_Rete_Sardegna_2021_06_03cr_AC_TF_-6.0_2.0_{}.npz'.format(N))
data = np.load(data_file, allow_pickle=True)
# names of all the variables for which we have some transfer function
all_var_names = list(data['var_names'])
# names of all the loads that were used as inputs in the computation of the transfer functions
all_load_names = list(data['load_names'])
# power flow solution
PF = data['PF'].item()

In [ ]:
# names of the loads that we want to use as input
load_names = ['EqX_MIMC_I2201TR1_____LOAD____', 'EqX_ALIC_I1501TR1_____LOAD____']
N_loads = len(load_names)
loads_idx = [all_load_names.index(load_name) for load_name in load_names]
# power absorbed by the loads
P_loads = np.array([PF['loads'][name]['P'] for name in load_names]) # [MW]
# figure labels
labels = ['{}: {:.1f} MW'.format(name.split('__')[0][4:9], P) for name,P in zip(load_names, P_loads)]

In [ ]:
# 's:xspeed'
# 'Grid-CODCTI-CODCTI0201________SUBNET__-CODCTI0201GGR1____GEN_____.ElmSym.speed'

# 'm:fe'
# 'Grid-BNFC_I-BNFC_I0601________SUBNET__-BNFC_I0601A1______BUS_____.ElmTerm.fe'
# 'Grid-CODCTI-CODCTI3801________SUBNET__-CODCTI3801B1______BUS_____.ElmTerm.fe'

# 'm:ur'
# 'Grid-BNFC_I-BNFC_I0601________SUBNET__-BNFC_I0601A1______BUS_____.ElmTerm.ur'
# 'Grid-CODCTI-CODCTI3801________SUBNET__-CODCTI3801B1______BUS_____.ElmTerm.ur'

var_types = ['m:ur','m:ur']
# names of the "output" variables of which we want to compute the PSD
var_names = ['Grid-NARCDI-NARCDI1501________SUBNET__-NARCDI1501A5______BUS_____.ElmTerm.ur',
             'Grid-BDNC_I-BDNC_I1501________SUBNET__-BDNC_I1501A2______BUS_____.ElmTerm.ur']
N_vars = len(var_names)
vars_idx = [all_var_names.index(var_name) for var_name in var_names]

titles = []
for i in range(N_vars):
    title = var_names[i].split('-')[-1].split('.')[0].split('__')[0]
    title += '.' + '.'.join(var_names[i].split('.')[-2:])
    titles.append(title)

In [ ]:
N_freq_samples,N_all_loads,N_all_vars = data['TF'].shape
print('The shape of the TF matrix is {}x{}x{} (# of frequency samples by # of loads by # of variables).'.\
      format(N_freq_samples,N_all_loads,N_all_vars))

In the following, we must use meshgrid to create indexing matrixes, since the variables `loads_idx` and `vars_idx` are not slices. Using

`TF = data['TF'][:, loads_idx, vars_idx]`

would only work if `loads_idx` and `vars_idx` had the same shape and in any case would not return the subset of rows and columns expected. See [here](https://numpy.org/doc/stable/user/basics.indexing.html#integer-array-indexing) for a thorough explanation of indexing on ndarrays.

In [ ]:
LOADS_IDX,VARS_IDX = np.meshgrid(loads_idx, vars_idx, indexing='ij')
TF = data['TF'][:, LOADS_IDX, VARS_IDX]

In [ ]:
OUT = data['OUT']
F = data['F']
F0 = 50.
out_multi = combine_output_spectra(OUT, load_names, var_names, all_load_names, all_var_names,
                                   var_types, F, PF, data['bus_equiv_terms'].item())

In [ ]:
fig,ax = plt.subplots(N_vars, 1, figsize=(5,2.5*N_vars), squeeze=False)
ax = ax[:,0]
cmap = plt.get_cmap('viridis', N_loads)
use_dBs = False
loc = 'lower left' if use_dBs else 'upper left'

for i in range(N_vars):
    y = np.abs(out_multi[i])
    if use_dBs:
        y = dB*np.log10(y)
    ax[i].semilogx(F, y, 'k', lw=2, label='Total')
    ax[i].set_ylabel('PSD [dB]')
    ax[i].set_title(titles[i])
    for j in range(N_loads):
        y = np.abs(OUT[:,loads_idx[j],vars_idx[i]])
        if use_dBs:
            y = dB*np.log10(y)
        ax[i].semilogx(F, y, color=cmap(j), lw=1, label=labels[j])
ax[-1].set_xlabel('Frequency [Hz]')
ax[0].legend(loc=loc, frameon=False, fontsize=9)
sns.despine()
fig.tight_layout()

Performs the vector fitting for a given number of poles:

In [ ]:
def run_vf(X, F, n_poles, n_iter=3, weights=None, poles_guess=None, do_plot=False):
    Y = X.astype(np.complex128)
    if weights is None:
        weights = np.ones(F.size, dtype=np.float64)
    else:
        assert weights.size == F.size
        weights = weights.astype(np.float64)

    F0,F1 = np.log10(F[[0,-1]])
    s = (2j*np.pi*F).astype(np.complex128)

    import vectfit3 as vf
    opts = vf.opts.copy()
    opts['asymp'] = 2
    opts['skip_res'] = True  # skip residue computation
    opts['spy2'] = False     # do not plot the results

    # initial guess for pole positions
    if poles_guess is not None:
        poles = poles_guess
    else:
        # logarithmically evenly spaced poles in the range [F0,F1]
        poles = -2*np.pi*np.logspace(F0, F1, n_poles, dtype=np.complex128)
    for i in range(n_iter):
        if i == n_iter-1:
            opts['skip_res'] = False
            opts['spy2'] = do_plot
        SER,poles,rmserr,fit = vf.vectfit(Y, s, poles, weights, opts)
    return SER,poles,rmserr,fit

Choose the number of poles based on a threshold of the RMS error:

In [ ]:
shp = N_vars,N_loads
N_poles = np.zeros(shp, dtype=int)
rms_err = np.zeros(shp)
rms_thresh = np.zeros(shp)
fit = np.zeros((N_vars, N_loads, F.size), dtype=complex)
systems = [[] for _ in range(N_vars)]
max_N_poles = 50
for i in range(N_vars):
    for j in range(N_loads):
        tf = TF[:,j,i]
        rms_thresh[i,j] = 10 ** (np.floor(np.log10(np.abs(tf).mean())) - 3)
        for n in range(max_N_poles):
            SER,_,rms_err[i,j],fit[i,j,:] = run_vf(tf, F, n+1)
            if abs(rms_err[i,j]) < rms_thresh[i,j]:
                break
        N_poles[i,j] = n+1
        systems[i].append(lti(SER['A'],SER['B'],SER['C'],SER['D']))
        print('[{:2d}][{:2d}] # of poles sufficient to have an RMS error below {:g}: {}.'.\
              format(i+1, j+1, rms_thresh[i,j], N_poles[i,j]))

The results of the vector fitting:

In [ ]:
fig,ax = plt.subplots(N_vars, 1, figsize=(5,2.5*N_vars), squeeze=False)
ax = ax[:,0]
for i in range(N_vars):
    for j in range(N_loads):
        tf = TF[:,j,i]
        if np.array(cmap(j))[:3].mean() > 0.5:
            col = 'k'
        else:
            col = 'w'
        ax[i].plot(F, dB*np.log10(np.abs(tf)), color=cmap(j), lw=3, label=labels[j])
        ax[i].plot(F, dB*np.log10(np.abs(fit[i,j])), '--', color=col, lw=1)
    ax[i].set_xscale('log')
    ax[i].set_ylabel('PSD [dB]')
ax[-1].set_xlabel('Frequency [Hz]')
ax[-1].legend(loc='lower left', frameon=False, fontsize=9)
sns.despine()
fig.tight_layout()

Generate an OU process with the appropriate statistics:

In [ ]:
tend = 12000
srate = 200
dt = 1/srate
mean,stddev,tau = 0,0.1*P_loads,20e-3
μ,c,α = mean,stddev*np.sqrt(2/tau),1/tau
cutoff = α/(2*np.pi)
time = np.r_[0 : tend+dt/2 : dt]
N_samples = time.size
U = np.zeros((N_loads,N_samples))
for i in tqdm(range(N_loads)):
    U[i,:] = OU_2(dt, α, μ, c[i], N_samples)

Filter the OU process with the TFs extracted above:

In [ ]:
Y = np.zeros((N_vars,N_samples))
for i in tqdm(range(N_vars)):
    ys = []
    for j in range(N_loads):
        _,y,_ = lsim(systems[i][j], U[j,:], time)
        assert y.imag.max() < 1e-10
        ys.append(y.real)
    Y[i,:] = np.sum(ys,axis=0)

Compute the PSDs:

In [ ]:
window = 200 / dt
onesided = True
def run_welch(x, dt, window, onesided):
    freq,P = welch(x, 1/dt, window='hamming',
                   nperseg=window, noverlap=window/2,
                   return_onesided=onesided, scaling='density')
    if onesided:
        P /= 2
    else:
        Nf = freq.size
        freq = freq[:Nf//2]
        P = P[:Nf//2]
    return freq, P, np.sqrt(P)

freq,P_U,abs_U = run_welch(U, dt, window, onesided)
P_U_dB = dB*np.log10(P_U)
abs_U_dB = dB*np.log10(abs_U)

_,P_Y,abs_Y = run_welch(Y, dt, window, onesided)
P_Y_dB = dB*np.log10(P_Y)
abs_Y_dB = dB*np.log10(abs_Y)

P_U_theor = np.array([(ci/α)**2 / (1 + (2*np.pi*F/α)**2) for ci in c])
P_U_theor_dB = dB*np.log10(P_U_theor)
abs_U_theor = np.sqrt(P_U_theor)
abs_U_theor_dB = dB*np.log10(abs_U_theor)

# abs_TFxU = np.abs(TF[:,:,IDX]) * abs_U_theor
# P_TFxU = abs_TFxU**2
# abs_TFxU_dB = dB*np.log10(abs_TFxU)
# P_TFxU_dB = dB*np.log10(P_TFxU)

Plot the PSDs:

In [ ]:
fig,ax = plt.subplots(N_vars+1, 1, figsize=(5,2.5*(N_vars+1)), sharex=True)

for i in range(N_loads):
    ax[0].plot(freq, abs_U[i,:], color=cmap(i), lw=0.75, label=labels[i])
    ax[0].plot(F, abs_U_theor[i,:], color='k', lw=2)
ax[0].legend(loc='lower left', frameon=False)

for i in range(N_vars):
    col = np.array(cmap(i)[:3]) + 0.3
    col[col>1] = 1
    ax[i+1].plot(freq, np.abs(abs_Y[i,:]), color=[.6,.6,.6], lw=1)
    for j in range(N_loads):
        ax[i+1].plot(F, np.abs(OUT[:,loads_idx[j],vars_idx[i]]), color='k', lw=1)
    ax[i+1].plot(F, np.abs(out_multi[i,:]), color='r', lw=1)
    ax[i+1].set_title(titles[i])

for a in ax:
    a.set_xscale('log')
    a.set_ylabel('| (j$\omega$)|')
ax[-1].set_xlabel('Frequency [Hz]')

sns.despine()
fig.tight_layout()